## 二値分類での近似
入力層から中間層への積の計算はEmbeddingレイヤを導入することにより単純に行列から行を抜き出すだけの軽量な処理となった。

次は中間層以降の問題、すなわち中間層のニューロンと重み行列の積とsoftmaxレイヤの計算に対応していく。

まず、最初の問題だが、これまでは出力層は全ての単語について確率を計算するため、中間層のニューロンに巨大な行列をかけていた。今回は全ての単語ではなく、ターゲットの単語に対応する行列の列ベクトルのみをかけて、確率を計算する。すなわち多値分類だったのを二値分類問題で近似する。

多値分類のときはsoftmax関数で確率に変換したが、二値分類のときはsigmoid関数を用いる

$$
y = \frac{1}{1 + \exp(-x)}
$$

誤差関数については多値も二値も交差エントロピー誤差を用いる。

$$
L = -(t\log y + (1-t)\log(1-y))
$$

行列から列を抜き出す操作はEmbeddingレイヤで実装した。抜き出す以外に列と中間層からの出力の内積を取る処理が必要だ。なのでこれも併せてEmbeddingDotレイヤを実装する

In [1]:
import sys
sys.path.append("../../deep-learning-from-scratch-2")
import numpy as np

class EmbeddingDot:
    def __init__(self, W):
        self.embed = Embedding(W)
        self.params = self.embed.params
        self.grads = self.embed.grads
        self.cache = None
        
    def forward(self, h, idx):
        target_W = self.embed.forward(idx)
        out = np.sum(target_W * h, axis=1)
        self.cache = (h, target_W)
        return out
    
    def backward(f, dout):
        h, target_W = self.cache
        dout = dout.reshape(dout.shape[0], 1)
        dtarget_W = dout * h
        self.embed.backward(dtarget_W)
        dh = dout * target_W
        return dh

## Negative Sampling
二値分類で正解ラベルになる確率を損失としたが、これでは不正解になるパターンを学習できない。すなわち、不正解ラベルに対しては確率を０に近づけるように学習していく必要がある。しかし、全ての不正解パターンについて計算すると二値にした意味が無いので不正解ラベル全体からいくつかサンプリングして学習を行う。

では、サンプリングをランダムに行ってもいいかと言うと、実はそれよりも良い方法が知られている。それは出現回数のによる確率分布に従う確率でサンプリングすること。すなわち、よく出現する単語はサンプリングされやすく、レアは単語はあまりサンプリングされないようにする。

ある確率分布に従うようにサンプリングするには、numpyのchoiceメソッドを使う

In [1]:
import numpy as np
# 0-9のランダムな整数
np.random.choice(10)

3

In [2]:
# リストからランダムに選択
words = ["apple", "banana", "pen", "cola"]
np.random.choice(words)

'cola'

In [3]:
# 確率分布に従ってサンプリング
p = [0.5, 0.25, 0.2, 0.05]
np.random.choice(words, p=p)

'apple'

In [4]:
np.random.choice(words, p=p)

'cola'

In [5]:
np.random.choice(words, p=p)

'apple'

In [6]:
np.random.choice(words, p=p)

'apple'

In [7]:
np.random.choice(words, p=p)

'banana'

実際word2vecで使用されているサンプリング手法では、もうひと手間加えられている
$$
P'(w_i) = \frac{P(w_i)^{0.75}}{\sum^n_j(P(w_i)^{0.75})}
$$
このようにすることで、分布全体を「緩めて」、確率の低い単語に対して確率を少しだけ高める。

これらの処理を含めてNegativeSamplingを行うクラスをUnigramSamplerクラスとしてch04/negative_sampling_layer.pyに実装してある。

In [8]:
import sys
sys.path.append("../../deep-learning-from-scratch-2")
from common.layers import SigmoidWithLoss
from ch04.negative_sampling_layer import UnigramSampler

class NegativeSamplingLoss:
    def __init__(self, W, corpus, power=0.75, sample_size = 5):
        self.sample_size=sample_size
        self.sampler = UnigramSampler(corpus, power, sample_size)
        self.loss_layers = [SigmoidWithLoss() for _ in range(sample_size + 1)]
        self.embed_dot_layers = [EmbeddingDot(W) for _ in range(sample_size + 1)]
        self.params, self.grads = [], []
        
        for layer in self.embed_dot_layers:
            self.params += layer.params
            self.grads += layer.grads
        
    def forward(self, h, target):
        batch_size = target.shape[0]
        negative_sample = self.sampler.get_negative_sample(target)
        
        score = self.embed_dot_layers[0].forward(h, target)
        correct_label = np.ones(batch_size, dtype=np.int32)
        loss = self.loss_layers[0].forward(score, correct_label)
    
        negative_label = np.zeros(batch_size, dtype=np.int32)
        for i in range(self.sample_size):
            negative_target = negative_sample[:, i]
            score = self.embed_dot_layers[1 + i].forward(h, negative_target)
            loss += self.loss_layers[1 + i].forward(score, negative_label)
        
        return loss
    
    def backward(self, dout=1):
        dh = 0
        for l0, l1 in zip(self.loss_layers, self.embed_dot_layers):
            dscore = l0.backward(dout)
            dh += l1.backward(dscore)
        
        return dh